In [4]:
from data_prep import *
from charts import *
from video_analysis import *
from team_sheets import *

# Load data
game_df = team_sheets()
players_df = players(game_df)
players_agg_df = players_agg(players_df)
lineouts_df = lineouts()
pitchero_df = pitchero_stats()
set_piece_df = set_piece_results()
analysis = game_stats()
 
# # Save data
game_df.to_csv('data/game.csv', index=False)
players_df.to_csv('data/players.csv', index=False)
players_agg_df.to_csv('data/players_agg.csv', index=False)
lineouts_df.to_csv('data/lineouts.csv', index=False)
pitchero_df.to_csv('data/pitchero.csv', index=False)
set_piece_df.to_csv('data/set_piece.csv', index=False)
analysis.to_csv('data/analysis.csv', index=False)
update_season_summaries(game_df)

# One-off charts (only source data needs updating)
# captains_chart(file='Charts/captains.html')
# results_chart(file='Charts/results.html')
# plot_games_by_player(file='Charts/appearances.html')
# plot_starts_by_position(file='Charts/positions.html')
# card_chart(file='Charts/cards.html')
# points_scorers_chart(file='Charts/points.html')
# team_sheets_chart(file='Charts/team-sheets.html')
# set_piece_h2h_chart(file='Charts/set-piece.html')

# Self-contained charts (chart needs updating)
game_stats_charts(analysis, file='Charts/video_analysis.html')
lineout_success(types=types, file='Charts/lineouts.html')

TypeError: update_season_summaries() missing 1 required positional argument: 'seasons'

In [2]:
game_df = team_sheets()
update_season_summaries(game_df,seasons)

### Season Summaries

In [30]:
# For each squad and season, calculate the following:
# - Number of games played
# - Number of players used
# - Player with most TotalGames (and the number of games)
#   If there are multiple players with the same number of games, join them with a '/'
df = (
    players_agg_df
    .sort_values("TotalGames", ascending=False)
    .groupby(["Season", "Squad"])
    .agg(
        GamesPlayed=("TotalGames", "count"),
        PlayersUsed=("Player", "nunique"),
        TotalGames=("TotalGames", "max"),
    )
    .reset_index()
)

top_players = players_agg_df.groupby(["Season", "Squad", "TotalGames"]).agg(
    Players=("Player", lambda x: " / ".join(x))
).reset_index()

df = df.merge(top_players, on=["Season", "Squad", "TotalGames"], how="left")
df

,Season,Squad,GamesPlayed,PlayersUsed,TotalGames,Players
0,2021/22,1st,61,61,19,Ryan Morlen / Max Crawley-Moore
1,2021/22,2nd,84,84,18,Harrison Berry
2,2022/23,1st,41,41,18,Ben Tottman / John Peaty / Max Crawley-Moore
3,2022/23,2nd,73,73,12,Harrison Berry
4,2023/24,1st,55,55,25,Ben Tottman / Ryland Thomas / Guy Collins
5,2023/24,2nd,81,81,14,Harrison Berry
6,2024/25,1st,50,50,18,Sam Lindsay-McCall
7,2024/25,2nd,86,86,15,Harrison Berry


,Squad,Season,Player,CupStarts,CupBench,LeagueStarts,LeagueBench,FriendlyStarts,FriendlyBench,CompetitiveStarts,CompetitiveBench,TotalStarts,TotalBench,TotalGames,MostCommonPosition,MostCommonPositionType
0,1st,2021/22,Josh Brimecombe,0.0,1.0,3.0,3.0,5.0,0.0,3.0,4.0,8.0,4.0,12,Prop,Forwards
1,1st,2022/23,Guy Collins,0.0,0.0,10.0,7.0,0.0,0.0,10.0,7.0,10.0,7.0,17,Prop,Forwards
2,1st,2024/25,Dave Bridges,0.0,0.0,3.0,4.0,0.0,0.0,3.0,4.0,3.0,4.0,7,Prop,Forwards
3,2nd,2021/22,Sam Lindsay,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,2,Prop,Forwards
4,2nd,2023/24,Rob Salvi,0.0,0.0,2.0,6.0,1.0,1.0,2.0,6.0,3.0,7.0,10,Prop,Forwards
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,2nd,2024/25,Ed Arundell,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,2,None,None
527,2nd,2024/25,Sam Lindsay-McCall,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,None,None
528,1st,2021/22,Fin Edwards,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,2,None,None
529,2nd,2022/23,Jordan Hollis,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1,None,None


In [57]:
def set_piece_summaries(df):
    df_agg = df.groupby(["Season", "Squad", "SetPiece", "Metric"])["Count"].agg(["sum", "count"]).reset_index()

    # Replace "_l_" and "_s_" with "_"
    df_agg["Metric"] = df_agg["Metric"].str.replace("_l_", "_").str.replace("_s_", "_")

    df_agg = df_agg.pivot_table(
        index=["Season", "Squad", "SetPiece", "count"], 
        columns="Metric", values="sum", fill_value=0
    ).reset_index().rename_axis(None, axis=1)

    df_agg["T_won"] = df_agg["Opp_lost"] / df_agg["count"]
    df_agg["T_lost"] = df_agg["EG_lost"] / df_agg["count"]
    df_agg["EG_total"] = (df_agg["EG_won"] + df_agg["EG_lost"]) / df_agg["count"]
    df_agg["Opp_total"] = (df_agg["Opp_won"] + df_agg["Opp_lost"]) / df_agg["count"]
    df_agg["EG_success"] = df_agg["EG_won"] / df_agg["EG_total"] / df_agg["count"]
    df_agg["Opp_success"] = df_agg["Opp_won"] / df_agg["Opp_total"] / df_agg["count"]

    df_agg = df_agg[[
        "Season", 
        "Squad", 
        "SetPiece", 
        "count", 
        "EG_total", 
        "EG_success", 
        "Opp_total", 
        "Opp_success",
        "T_won",
        "T_lost"
    ]]

    # lineouts = df_agg[df_agg["SetPiece"] == "Lineout"].copy().drop(columns=["SetPiece"], inplace=False)
    # scrums = df_agg[df_agg["SetPiece"] == "Scrum"].copy().drop(columns=["SetPiece"], inplace=False)

    # wide_df = pd.merge(
    #     lineouts, 
    #     scrums, 
    #     on=["Season", "Squad"], 
    #     suffixes=("_lineout", "_scrum")
    # )

    # Convert to dictionary for easy JS manipulation
    d = {}
    for (x, y, z), group in df_agg.groupby(["Season", "Squad", "SetPiece"]):
        d.setdefault(x, {}).setdefault(y, {})[z] = group.drop(columns=["Season", "Squad", "SetPiece"]).to_dict(orient="records")[0]

    # Save to JSON
    with open("data/set_piece_summaries.json", "w") as f:
        json.dump(d, f, indent=4)

    return df_agg

set_piece_df = set_piece_results()
summary = set_piece_summaries(set_piece_df)

In [53]:
summary

,Season,Squad,count_lineout,EG_total_lineout,EG_success_lineout,Opp_total_lineout,Opp_success_lineout,T_won_lineout,T_lost_lineout,count_scrum,EG_total_scrum,EG_success_scrum,Opp_total_scrum,Opp_success_scrum,T_won_scrum,T_lost_scrum
0,2021/22,1st,9,10.888889,0.897959,10.555556,0.663158,3.555556,1.111111,9,6.555556,0.915254,8.111111,0.863014,1.111111,0.555556
1,2021/22,2nd,8,12.000000,0.770833,10.625000,0.600000,4.250000,2.750000,8,8.000000,0.843750,10.125000,0.888889,1.125000,1.250000
2,2022/23,1st,17,10.411765,0.847458,10.941176,0.736559,2.882353,1.588235,17,8.411765,0.944056,7.647059,0.876923,0.941176,0.470588
3,2022/23,2nd,3,14.000000,0.714286,6.000000,0.722222,1.666667,4.000000,3,8.000000,0.958333,7.000000,0.952381,0.333333,0.333333
4,2023/24,1st,21,13.857143,0.697595,11.380952,0.799163,2.285714,4.190476,21,8.857143,0.919355,7.761905,0.766871,1.809524,0.714286
5,2023/24,2nd,7,10.857143,0.855263,9.000000,0.682540,2.857143,1.571429,7,6.857143,0.770833,8.714286,0.819672,1.571429,1.571429
6,2024/25,1st,14,11.500000,0.695652,11.714286,0.841463,1.857143,3.500000,14,8.428571,1.000000,7.214286,0.811881,1.357143,0.000000
7,2024/25,2nd,9,11.000000,0.666667,10.666667,0.739583,2.777778,3.666667,9,6.888889,0.838710,9.000000,0.901235,0.888889,1.111111


In [ ]:
summary.pivot_table(index=["Season", "Squad"], columns="SetPiece", values=["EG_success",], fill_value=0)

EG_success           Opp_success          
SetPiece         Lineout     Scrum     Lineout     Scrum
Season  Squad                                           
2021/22 1st     0.897959  0.915254    0.663158  0.863014
        2nd     0.770833  0.843750    0.600000  0.888889
2022/23 1st     0.847458  0.944056    0.736559  0.876923
        2nd     0.714286  0.958333    0.722222  0.952381
2023/24 1st     0.697595  0.919355    0.799163  0.766871
        2nd     0.855263  0.770833    0.682540  0.819672
2024/25 1st     0.695652  1.000000    0.841463  0.811881
        2nd     0.666667  0.838710    0.739583  0.901235

In [33]:
# Summary to dict {"2021/22": {"1st XV": {"Lineout": {"EG_total": 10, "EG_success": 0.8, ...}, "2nd XV": {}}, "2022/23":{...}}
# 
# Convert to dictionary format
d = {}
for (x, y), group in summary.groupby(["Season", "Squad"]):
    d.setdefault(x, {})[y] = group.drop(columns=["Season", "Squad"]).to_dict(orient="records")

# Save to JSON
with open("data/set_piece_summaries.json", "w") as f:
    json.dump(d, f, indent=4)


In [36]:

html_output = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
</head>
<body class="container mt-4">
    <h2 class="text-center">Rugby Season Summary</h2>
    <div class="mb-3">
        <label for="seasonSelect" class="form-label">Select Season:</label>
        <select id="seasonSelect" class="form-select" onchange="updateTable()">
            <option value="2024-25">2024-25</option>
            <option value="2023-24">2023-24</option>
            <option value="2022-23">2022-23</option>
            <option value="2021-22">2021-22</option>
        </select>
    </div>
    
    <div class="table-responsive">
        <table class="table table-borderless text-center align-middle">
                        <thead class="table-dark">
                            <tr>
                                <th rowspan="2">
                                    <select class="form-select" id="seasonSelect" onchange="updateTable()"
                                        style="width: min-content;">
                                        <option value="2024/25">2024/25</option>
                                        <option value="2023/24">2023/24</option>
                                        <option value="2022/23">2022/23</option>
                                        <option value="2021/22">2021/22</option>
                                    </select>
                                </th>
                                <th class="align-middle py-0 my-0" rowspan="2">Games</th>
                                <th class="align-middle py-0 my-0" colspan="2">EG</th>
                                <th class="align-middle py-0 my-0" colspan="2">Opposition</th>
                                <th class="align-middle py-0 my-0" colspan="2">Turnovers</th>
                            </tr>
                            <tr>
                                <th class="align-middle py-0">Average per game</th>
                                <th class="align-middle py-0 m-0">Success</th>
                                <th class="align-middle py-0">Average per game</th>
                                <th class="align-middle py-0 m-0">Success</th>
                                <th class="align-middle py-0">Won</th>
                                <th class="align-middle py-0 m-0">Lost</th>
                            </tr>
                        </thead>
                        <tbody>
                            <tr>
                                <td>1st XV</td>
                                <td>{scrum_agg["count"].iloc[0]}</td>
                                <td>{scrum_agg["EG_total"].iloc[0]:.1f}</td>
                                <td>{scrum_agg["EG_success"].iloc[0]:.0%}</td>
                                <td>{scrum_agg["Opp_total"].iloc[0]:.1f}</td>
                                <td>{scrum_agg["Opp_success"].iloc[0]:.0%}</td>
                                <td>{scrum_agg["T_won"].iloc[0]:.1f}</td>
                                <td>{scrum_agg["T_lost"].iloc[0]:.1f}</td>
                            </tr>
                            <tr>
                                <td>2nd XV</td>
                                <td>{scrum_agg["count"].iloc[1]}</td>
                                <td>{scrum_agg["EG_total"].iloc[1]:.1f}</td>
                                <td>{scrum_agg["EG_success"].iloc[1]:.0%}</td>
                                <td>{scrum_agg["Opp_total"].iloc[1]:.1f}</td>
                                <td>{scrum_agg["Opp_success"].iloc[1]:.0%}</td>
                                <td>{scrum_agg["T_won"].iloc[1]:.1f}</td>
                                <td>{scrum_agg["T_lost"].iloc[1]:.1f}</td>
                            </tr>

                        </tbody>
                    </table>
    </div>
</body>
</html>
"""

display(HTML(html_output))

{'1st XV': {'Played': 17.0,
  'Won': 3.0,
  'Lost': 14.0,
  'Avg_PF': 17.176470588235293,
  'Avg_PA': 31.352941176470587},
 '2nd XV': {'Played': 14.0,
  'Won': 7.0,
  'Lost': 7.0,
  'Avg_PF': 19.428571428571427,
  'Avg_PA': 22.642857142857142}}

In [ ]:
from IPython.core.display import display, HTML
import pandas as pd

# Calculate summary statistics
def generate_season_summary(df, season):
    season_data = df[df["Season"] == season]
    summary = season_data.groupby("Squad").agg(
        Played=("GameID", "count"),
        Won=("Result", lambda x: (x == "W").sum()),
        Lost=("Result", lambda x: (x == "L").sum()),
        Avg_PF=("PF", "mean"),
        Avg_PA=("PA", "mean"),
    ).reset_index()
    
    # Convert to dictionary for easy JS manipulation
    summary_dict = summary.set_index("Squad").T.to_dict()
    
    return summary_dict

# Generate dropdown options
seasons = game_df["Season"].unique()
dropdown_options = "".join([f'<option value="{s}">{s}</option>' for s in seasons[::-1]])

# Generate initial table for the first season
initial_season = seasons[-1]
initial_summary = generate_season_summary(game_df, initial_season)

# HTML Output
# Inject into existing index.html
with open("index.html", "r", encoding="utf-8") as f:
    index_html = f.read()


soup = BeautifulSoup(index_html, "html.parser")

# Update dropdown options
select = soup.find("select", id="seasonSelect")
select.clear()
for season in seasons[::-1]:
    option = soup.new_tag("option")
    option["value"] = season
    option.string = season
    select.append(option)

# Update table with initial summary
table = soup.find("table")
for squad, data in initial_summary.items():
    row = table.find("tr", id=f"row{squad[0]}")
    row.find("td", id=f"played{squad[0]}").string = str(int(data["Played"]))
    row.find("td", id=f"won{squad[0]}").string = str(int(data["Won"]))
    row.find("td", id=f"lost{squad[0]}").string = str(int(data["Lost"]))
    row.find("td", id=f"pf{squad[0]}").string = f"{data['Avg_PF']:.1f}"
    row.find("td", id=f"pa{squad[0]}").string = f"{data['Avg_PA']:.1f}"

# Save updated index.html
with open("index.html", "w", encoding="utf-8") as f:
    f.write(str(soup))


# html_output = f"""
# <!DOCTYPE html>
# <html lang="en">
# <head>
#     <meta charset="UTF-8">
#     <meta name="viewport" content="width=device-width, initial-scale=1.0">
#     <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
#     <title>Rugby Season Summary</title>
#     <script>
#         const summaries = {initial_summary};

#         function updateTable() {{
#             const selectedSeason = document.getElementById("seasonSelect").value;
#             fetch("data/" + selectedSeason + ".json")
#                 .then(response => response.json())
#                 .then(data => {{
#                     document.getElementById("played1").innerText = data["1st XV"].Played;
#                     document.getElementById("won1").innerText = data["1st XV"].Won;
#                     document.getElementById("lost1").innerText = data["1st XV"].Lost;
#                     document.getElementById("pf1").innerText = data["1st XV"].Avg_PF.toFixed(1);
#                     document.getElementById("pa1").innerText = data["1st XV"].Avg_PA.toFixed(1);
                    
#                     document.getElementById("played2").innerText = data["2nd XV"].Played;
#                     document.getElementById("won2").innerText = data["2nd XV"].Won;
#                     document.getElementById("lost2").innerText = data["2nd XV"].Lost;
#                     document.getElementById("pf2").innerText = data["2nd XV"].Avg_PF.toFixed(1);
#                     document.getElementById("pa2").innerText = data["2nd XV"].Avg_PA.toFixed(1);
#                 }});
#         }}
#     </script>
# </head>
# <body class="container mt-4">
#     <h2 class="text-center">Rugby Season Summary</h2>
#     <div class="mb-3">
#         <label for="seasonSelect" class="form-label">Select Season:</label>
#         <select id="seasonSelect" class="form-select" onchange="updateTable()">
#             {dropdown_options}
#         </select>
#     </div>
    
#     <div class="table-responsive">
#         <table class="table table-bordered text-center align-middle">
#             <thead class="table-dark">
#                 <tr>
#                     <th>Squad</th>
#                     <th>Played</th>
#                     <th>Won</th>
#                     <th>Lost</th>
#                     <th>Avg PF</th>
#                     <th>Avg PA</th>
#                 </tr>
#             </thead>
#             <tbody>
#                 <tr>
#                     <td class="table-primary">1st XV</td>
#                     <td id="played1">{initial_summary["1st XV"]["Played"]}</td>
#                     <td id="won1">{initial_summary["1st XV"]["Won"]}</td>
#                     <td id="lost1">{initial_summary["1st XV"]["Lost"]}</td>
#                     <td id="pf1">{initial_summary["1st XV"]["Avg_PF"]:.1f}</td>
#                     <td id="pa1">{initial_summary["1st XV"]["Avg_PA"]:.1f}</td>
#                 </tr>
#                 <tr>
#                     <td class="table-secondary">2nd XV</td>
#                     <td id="played2">{initial_summary["2nd XV"]["Played"]}</td>
#                     <td id="won2">{initial_summary["2nd XV"]["Won"]}</td>
#                     <td id="lost2">{initial_summary["2nd XV"]["Lost"]}</td>
#                     <td id="pf2">{initial_summary["2nd XV"]["Avg_PF"]:.1f}</td>
#                     <td id="pa2">{initial_summary["2nd XV"]["Avg_PA"]:.1f}</td>
#                 </tr>
#             </tbody>
#         </table>
#     </div>
# </body>
# </html>
# """

# display(HTML(html_output))

# # Save to file
# with open("season_summary.html", "w", encoding="utf-8") as f:
#     f.write(html_output)

# Save JSON summaries for JavaScript fetching
import json
for season in seasons:
    summary = generate_season_summary(game_df, season)
    with open(f"data/{season.replace('/','-')}.json", "w") as f:
        json.dump(summary, f, indent=4)


/var/folders/4s/jv95m3qx4lb8ys12l4pg308w0000gp/T/ipykernel_91865/1267223303.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


NameError: name 'game_df' is not defined

# Individual Player Stats

In [ ]:
def player_data(p):
    pitchero = pitchero_df[pitchero_df["Player"]==p]
    pdf = players_df[players_df["Player"]==p]
    p_agg = players_agg_df[players_agg_df["Player"]==p]
    
    return {
        "pitchero": pitchero,
        "pdf": pdf,
        "p_agg": p_agg
    }

a,b,c = player_data("Sam Lindsay-McCall").values()    

In [ ]:
p = "Dan Billin"

def squad_pie(p):
    base = (
        alt.Chart(players_agg_df).encode(
            theta=alt.Theta("sum(TotalGames)").stack(True),
            color=alt.Color("Squad:N", scale=squad_scale, legend=alt.Legend(title=None, labelExpr="datum.label + ' XV'"))
        )
        .transform_filter(f"datum.Player === '{p}'")
        .transform_calculate(label="datum.Squad + ' XV'")
    )

    pie = base.mark_arc(outerRadius=120, opacity=0.8)
    text1 = base.mark_text(radius=75, size=36).encode(
        theta=alt.Theta("sum(TotalGames)", stack=True),
        text=alt.Text("sum(TotalGames)"), 
        detail="Squad:N",
        color=alt.value("white")
    )
    text2 = base.mark_text(radius=150, size=24).encode(
        theta=alt.Theta("sum(TotalGames)", stack=True),
        text=alt.Text("label:N"),
        detail="Squad:N",
    )

    return pie + text1 + text2

position_order = ["Prop", "Hooker", "Second Row", "Back Row", "Scrum Half", "Fly Half", "Centre", "Back Three"]
position_color = ["#202947", "#146f14", "#981515", "#b03030"]

def position_pie(p):
    base = (
        alt.Chart(players_df)
        .transform_calculate(posi=f"indexof({position_order}, datum.Position)")
        .encode(
            theta=alt.Theta("count()").stack(True),
            color=alt.Color(
                "Position:N"
                legend=alt.Legend(title=None, orient="bottom", offset=40), 
                scale=alt.Scale()
            )
        )
        .transform_filter(f"datum.Player === '{p}' & isValid(datum.Position)")
    )

    pie = base.mark_arc(outerRadius=120)
    text = base.mark_text(radius=75, size=36).encode(
        theta=alt.Theta("count()", stack=True),
        text=alt.Text("count()"), 
        color=alt.value("white"),
        detail="Position:N"
    )

    return (pie + text).transform_filter(f"datum.Player === '{p}'")

def games(p):
    bar = (
        alt.Chart(players_agg_df).encode(
            x=alt.X("Date:T", title="Date"),
            y=alt.Y("count()", title="Games Played"),
            color=alt.Color("Position:N", scale=position_scale)
        )
        .transform_filter(f"datum.Player === '{p}'")
        .mark_bar()
    )

position_pie(p)
# squad_pie(p)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (2585547723.py, line 38)

In [ ]:
b

,Squad,Season,Competition,GameType,Opposition,Home/Away,PF,PA,Result,Captain,VC1,VC2,Number,Player,Position,PositionType
422,1st,2021/22,Friendly,Friendly,Metropolitan Police,H,29,28,W,Jack Andrews,<NA>,<NA>,4,Sam Lindsay-McCall,Second Row,Forwards
423,1st,2021/22,Friendly,Friendly,London Irish,A,10,33,L,Jack Andrews,Sam Lindsay-McCall,<NA>,4,Sam Lindsay-McCall,Second Row,Forwards
424,1st,2021/22,Friendly,Friendly,Horsham,A,26,56,L,Jack Andrews,James Funnell,<NA>,4,Sam Lindsay-McCall,Second Row,Forwards
425,1st,2021/22,Friendly,Friendly,Purley John Fisher,H,19,33,L,Jack Andrews,James Funnell,<NA>,4,Sam Lindsay-McCall,Second Row,Forwards
426,1st,2021/22,Sussex 1,League,Eastbourne,H,47,0,W,Jack Andrews,<NA>,<NA>,4,Sam Lindsay-McCall,Second Row,Forwards
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2289,1st,2023/24,Counties 1 Surrey/Sussex,League,Eastbourne,A,24,26,L,Ryland Thomas,Dan Poulton,<NA>,17,Sam Lindsay-McCall,NaN,Bench
2313,1st,2024/25,Counties 1 Surrey/Sussex,League,Eastbourne,A,36,33,W,Ryland Thomas,Chris May-Miller,<NA>,17,Sam Lindsay-McCall,NaN,Bench
2314,1st,2024/25,Counties 1 Surrey/Sussex,League,Trinity,H,14,48,L,Ryland Thomas,<NA>,<NA>,17,Sam Lindsay-McCall,NaN,Bench
2315,1st,2024/25,Counties 1 Surrey/Sussex,League,Old Rutlishians,H,19,23,L,Ryland Thomas,<NA>,<NA>,17,Sam Lindsay-McCall,NaN,Bench


# Team Sheets

In [ ]:
def team_sheet_chart(
        squad=1, 
        names=None, 
        captain=None, 
        vc=None, 
        opposition=None, 
        home=True, 
        competition="Counties 1 Sussex",
        season="2023/24"
    ):

    if names is None:
        df = team_sheets()    

        # Last row as dict
        team = df.iloc[-1].to_dict()


        label = f'{"1st" if squad==1 else "2nd"} XV vs {team["Opposition"]}({team["Home/Away"]})'
        captain = team["Captain"]
        vc = team["VC1"]
        season = team["Season"]
        competition = team["Competition"]

        # Keep keys that can be converted to integers
        team = {int(k): v for k, v in team.items() if k.isnumeric() and v}

        # Convert team to dataframe with Number and Player columns
        team = pd.DataFrame(team.items(), columns=["Number", "Player"])

    else:
        label = f'{"1st" if squad==1 else "2nd"} XV vs {opposition} ({"H" if home else "A"})'

        # Convert names to Player column of a dataframe with Number column (1-len(names))
        team = pd.DataFrame({"Player": names, "Number": range(1, len(names)+1)})

    coords = pd.DataFrame([
                {"n": 1, "x": 10, "y": 81},
                {"n": 2, "x": 25, "y": 81},
                {"n": 3, "x": 40, "y": 81},
                {"n": 4, "x": 18, "y": 69},
                {"n": 5, "x": 32, "y": 69},
                {"n": 6, "x": 6, "y": 61},
                {"n": 7, "x": 44, "y": 61},
                {"n": 8, "x": 25, "y": 56},
                {"n": 9, "x": 20, "y": 42},
                {"n": 10, "x": 38, "y": 36},
                {"n": 11, "x": 8, "y": 18},
                {"n": 12, "x": 56, "y": 30},
                {"n": 13, "x": 74, "y": 24},
                {"n": 14, "x": 92, "y": 18},
                {"n": 15, "x": 50, "y": 10},
                {"n": 16, "x": 80, "y": 82},
                {"n": 17, "x": 80, "y": 74},
                {"n": 18, "x": 80, "y": 66},
                {"n": 19, "x": 80, "y": 58},
                {"n": 20, "x": 80, "y": 50},
                {"n": 21, "x": 80, "y": 42},
                {"n": 22, "x": 80, "y": 34},
                {"n": 23, "x": 80, "y": 26},
            ])
    team = team.merge(coords, left_on="Number", right_on="n", how="inner").drop(columns="n")

    # Add captain (C) and vice captain (VC) else None
    team["Captain"] = team["Player"].apply(lambda x: "C" if x == captain else "VC" if x == vc else None)

    team["Player"] = team["Player"].str.split(" ")

    team.to_dict(orient="records")

    with open("team-sheet-lineup.json") as f:
        chart = json.load(f)
    chart["data"]["values"] = team.to_dict(orient="records")
    chart["title"]["text"] = label
    chart["title"]["subtitle"] = f"{season} - {competition}"

    n_replacements = len(team) - 15
    
    y = 126 + (n_replacements * 64)
    chart["layer"][0]["mark"]["y2"] = y
    # return chart
    return alt.Chart.from_dict(chart)

team_sheet_chart()

alt.LayerChart(...)

In [ ]:
pitchero_df[pitchero_df["Player"]=="Sam Lindsay-McCall"]

,Player,Season,Squad,TotalGames,Player_join,A,T,Con,PK,DG,YC,RC,Points,PPG,Tries,Cons,Pens,Cards
9,Sam Lindsay-McCall,2023/24,1st,23,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Sam Lindsay-McCall,2021/22,2nd,2,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,Sam Lindsay-McCall,2024/25,2nd,1,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,Sam Lindsay-McCall,2021/22,1st,15,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,Sam Lindsay-McCall,2024/25,1st,15,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270,Sam Lindsay-McCall,2022/23,2nd,1,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
470,Sam Lindsay-McCall,2022/23,1st,15,S Lindsay-Mccall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pitchero_df[pitchero_df["Player_join"]=="S Lindsay"].sort_values(["Season","Squad"])

,Player,Season,Squad,TotalGames,Player_join,A,T,Con,PK,DG,YC,RC,Points,PPG,Tries,Cons,Pens
464,Sam Lindsay-McCall,2021/22,1st,15,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
348,Sam Lindsay-McCall,2021/22,2nd,2,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Sam Lindsay-McCall,2022/23,1st,15,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,Sam Lindsay-McCall,2022/23,2nd,1,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
346,Sam Lindsay-McCall,2023/24,1st,23,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,Sam Lindsay-McCall,2024/25,1st,15,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,Sam Lindsay-McCall,2024/25,2nd,1,S Lindsay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
